# MMAD Dataset – Full Structure & EDA

이 노트북은 **MMAD 데이터셋의 전체 구조를 이해**하기 위한 목적의 EDA입니다.

### 이 노트북에서 다루는 것
1. `mmad.json` **전체 스키마 구조 정리**
2. 데이터셋 **폴더 트리 자동 탐색**
3. 질문 타입 / 옵션 / shot(template) 구조 통계
4. mask 존재 여부 및 활용 가능성
5. 실험 설계를 위해 반드시 필요한 **핵심 인사이트 요약**


In [1]:
from pathlib import Path
import os, json
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
from PIL import Image
import matplotlib.pyplot as plt


## 0. Dataset 경로 설정

In [2]:
MMAD_ROOT = Path(os.environ.get("MMAD_ROOT", "/Volumes/T7/Dataset/MMAD")).expanduser()
MMAD_JSON = MMAD_ROOT / "mmad.json"
print("MMAD_ROOT:", MMAD_ROOT)
print("mmad.json exists:", MMAD_JSON.exists())


MMAD_ROOT: /Volumes/T7/Dataset/MMAD
mmad.json exists: True


## 1. mmad.json 로드 및 Top-level 구조

In [3]:
with MMAD_JSON.open("r", encoding="utf-8") as f:
    mmad = json.load(f)

print("Total images:", len(mmad))
sample_key = next(iter(mmad.keys()))
print("Sample key:", sample_key)
print("Fields:", mmad[sample_key].keys())


Total images: 8366
Sample key: DS-MVTec/bottle/image/broken_large/000.png
Fields: dict_keys(['image_path', 'conversation', 'mask_path', 'similar_templates', 'random_templates'])


### 1.1 mmad.json 기본 스키마 설명

- `image_path`
- `conversation` (객관식 MCQ)
- `mask_path`
- `similar_templates`
- `random_templates`


## 2. Conversation 구조 분석

In [4]:
def parse_conversation_stats(mmad):
    rows = []
    for k, v in mmad.items():
        for q in v.get("conversation", []):
            rows.append({
                "image": k,
                "type": q.get("type"),
                "num_options": len(q.get("Options", {})),
                "has_annotation": q.get("annotation", False)
            })
    return pd.DataFrame(rows)

conv_df = parse_conversation_stats(mmad)
conv_df["type"].value_counts()


type
Anomaly Detection        8297
Defect Localization      4878
Defect Analysis          4782
Defect Description       4710
Defect Classification    4688
Object Classification    3155
Object Analysis          3084
Object Structure         3052
Object Details           3024
Name: count, dtype: int64

## 3. Shot(Template) 구조

In [5]:
tmpl_df = pd.DataFrame([
    {
        "num_similar": len(v.get("similar_templates", [])),
        "num_random": len(v.get("random_templates", []))
    }
    for v in mmad.values()
])
tmpl_df.describe()


,num_similar,num_random
count,8366.0,8366.0
mean,8.0,8.0
std,0.0,0.0
min,8.0,8.0
25%,8.0,8.0
50%,8.0,8.0
75%,8.0,8.0
max,8.0,8.0


## 4. Mask 구조

In [6]:
mask_df = pd.DataFrame([
    {"has_mask": bool(v.get("mask_path"))}
    for v in mmad.values()
])
mask_df["has_mask"].value_counts()


has_mask
True     5076
False    3290
Name: count, dtype: int64

## 5. 폴더 트리 (상위)

In [7]:
def print_tree(path: Path, depth=2, prefix=""):
    if depth < 0: return
    for p in sorted(path.iterdir()):
        print(prefix + ("📁 " if p.is_dir() else "📄 ") + p.name)
        if p.is_dir():
            print_tree(p, depth-1, prefix+"  ")

print_tree(MMAD_ROOT, depth=3)


📄 ._DS-MVTec
📄 ._DS-MVTec.zip
📄 ._GoodsAD
📄 ._GoodsAD.zip
📄 ._MVTec-AD
📄 ._MVTec-AD.zip
📄 ._MVTec-LOCO
📄 ._MVTec-LOCO.zip
📄 ._README.md
📄 ._VisA.zip
📄 ._domain_knowledge.json
📄 ._mmad.json
📁 .git
  📄 ._config
  📄 ._lfs
  📄 HEAD
  📄 config
  📄 description
  📁 hooks
    📄 ._post-checkout
    📄 ._post-commit
    📄 ._post-merge
    📄 ._pre-push
    📄 applypatch-msg.sample
    📄 commit-msg.sample
    📄 fsmonitor-watchman.sample
    📄 post-checkout
    📄 post-commit
    📄 post-merge
    📄 post-update.sample
    📄 pre-applypatch.sample
    📄 pre-commit.sample
    📄 pre-merge-commit.sample
    📄 pre-push
    📄 pre-push.sample
    📄 pre-rebase.sample
    📄 pre-receive.sample
    📄 prepare-commit-msg.sample
    📄 push-to-checkout.sample
    📄 sendemail-validate.sample
    📄 update.sample
  📄 index
  📁 info
    📄 exclude
  📁 lfs
    📄 ._incomplete
    📄 ._objects
    📄 ._tmp
    📁 incomplete
    📁 objects
      📄 ._0a
      📄 ._1b
      📄 ._63
      📄 ._d8
      📄 ._e2
      📄 ._f8
      📁 0a
   

In [26]:
# ============================
# [PASTE CELL] Load + Analyze full mmad.json
# - 목적: 전체 mmad.json 로드 후 구조/분포/경로 sanity check까지 한 번에
# - 사용법: MMAD_ROOT만 본인 환경으로 수정 후 실행
# ============================

from __future__ import annotations

import os, json, random
from pathlib import Path
from collections import Counter, defaultdict

import pandas as pd
import numpy as np
from PIL import Image

# ======= 1) 경로 설정 (여기만 수정) =======
MMAD_ROOT = Path(os.environ.get("MMAD_ROOT", "/Volumes/T7/Dataset/MMAD")).expanduser()
MMAD_JSON = MMAD_ROOT / "mmad.json"
# =========================================

assert MMAD_JSON.exists(), f"mmad.json not found: {MMAD_JSON}"
print("✅ MMAD_ROOT:", MMAD_ROOT)
print("✅ MMAD_JSON :", MMAD_JSON)

# ======= 2) 로드 =======
with MMAD_JSON.open("r", encoding="utf-8") as f:
    mmad = json.load(f)

print(f"✅ loaded mmad.json | num_images={len(mmad):,}")

# ======= 3) 스키마/샘플 확인 =======
sample_key = next(iter(mmad.keys()))
sample = mmad[sample_key]
print("\n--- Sample Key ---")
print(sample_key)
print("--- Fields ---")
print(list(sample.keys()))
print("--- 1st conversation item ---")
print(sample.get("conversation", [None])[0])

# ======= 4) 질문(Conversation) flatten =======
def normalize_options(opts):
    # Options는 보통 dict지만 혹시 다른 타입일 때도 안전 처리
    if isinstance(opts, dict):
        return {str(k).strip(): str(v).strip() for k, v in opts.items()}
    if isinstance(opts, list):
        out = {}
        for i, v in enumerate(opts):
            out[chr(ord("A")+i)] = str(v).strip()
        return out
    return {}

q_rows = []
img_rows = []

for k, v in mmad.items():
    img_rows.append({
        "key": k,
        "image_path": '/'.join(k.split('/')[:2]) + '/' + v.get("image_path"), 
        "mask_path": '/'.join(k.split('/')[:2]) + '/' + v.get("mask_path") if v.get("mask_path") else None,
        "num_similar_templates": len(v.get("similar_templates", []) or []),
        "num_random_templates": len(v.get("random_templates", []) or []),
        "num_questions": len(v.get("conversation", []) or []),
    })

    for q in (v.get("conversation", []) or []):
        opts = normalize_options(q.get("Options") or q.get("options") or {})
        q_rows.append({
            "key": k,
            "question_type": q.get("type", "unknown"),
            "question": q.get("Question") or q.get("question") or "",
            "answer": str(q.get("Answer") or q.get("answer") or "").strip(),
            "num_options": len(opts),
            "options_keys": "".join(sorted(opts.keys())),
            "annotation": bool(q.get("annotation", False)),
        })

img_df = pd.DataFrame(img_rows)
q_df   = pd.DataFrame(q_rows)

print("\n✅ DataFrames created")
print(" - img_df:", img_df.shape)
print(" - q_df  :", q_df.shape)

display(img_df.head(3))
display(q_df.head(5))

# ======= 5) 핵심 분포 통계 =======
print("\n===== (A) 질문 타입 분포 =====")
display(q_df["question_type"].value_counts())

print("\n===== (B) 옵션 개수 분포 =====")
display(q_df["num_options"].value_counts().sort_index())

print("\n===== (C) 이미지당 질문 개수 분포 =====")
display(img_df["num_questions"].value_counts().sort_index())

print("\n===== (D) template 개수 분포 (similar/random) =====")
display(img_df[["num_similar_templates","num_random_templates"]].value_counts().head(20))

print("\n===== (E) mask 존재 여부 =====")
display(img_df["mask_path"].notna().value_counts())

# ======= 6) (선택) 경로 sanity check: 이미지/마스크/템플릿 존재 여부 =======
SAMPLE_N = 100  # 너무 크면 느려질 수 있으니 샘플링
keys = list(mmad.keys())
random.seed(42)
keys = random.sample(keys, min(SAMPLE_N, len(keys)))

def exists_rel(rel: str | None) -> bool:
    if not rel or str(rel).lower() in ("none","nan","null"):
        return False
    return (MMAD_ROOT / rel).exists()

path_checks = []
for k in keys:
    v = mmad[k]
    img_rel = '/'.join(k.split('/')[:2]) + '/' + v.get("image_path")
    mask_rel = '/'.join(k.split('/')[:2]) + '/' + v.get("mask_path") if v.get("mask_path") else None
    sim = (v.get("similar_templates") or [])
    rnd = (v.get("random_templates") or [])

    path_checks.append({
        "key": k,
        "img_rel": img_rel,
        "img_exists": exists_rel(img_rel),
        "mask_rel": mask_rel,
        "mask_exists": exists_rel(mask_rel),
        "sim0": sim[0] if len(sim)>0 else None,
        "sim0_exists": exists_rel(sim[0]) if len(sim)>0 else False,
        "rnd0": rnd[0] if len(rnd)>0 else None,
        "rnd0_exists": exists_rel(rnd[0]) if len(rnd)>0 else False,
    })

check_df = pd.DataFrame(path_checks)
print("\n===== (F) 경로 존재 여부(샘플) =====")
display(check_df[["img_exists","mask_exists","sim0_exists","rnd0_exists"]].mean().rename("exist_ratio"))

# ======= 7) (선택) 이미지 해상도 분포(샘플) =======
# 이미지가 크면 느릴 수 있으니 샘플 기반
res_rows = []
for _, r in check_df.iterrows():
    if not r["img_exists"]:
        continue
    p = MMAD_ROOT / r["img_rel"]
    try:
        with Image.open(p) as im:
            w, h = im.size
        res_rows.append((w, h))
    except Exception:
        pass

if res_rows:
    res_ser = pd.Series(res_rows).value_counts().head(20)
    print("\n===== (G) 이미지 해상도 TOP =====")
    display(res_ser)
else:
    print("\n(G) 이미지 해상도: 샘플에서 로드 실패(경로/포맷 확인 필요)")

✅ MMAD_ROOT: /Volumes/T7/Dataset/MMAD
✅ MMAD_JSON : /Volumes/T7/Dataset/MMAD/mmad.json
✅ loaded mmad.json | num_images=8,366

--- Sample Key ---
DS-MVTec/bottle/image/broken_large/000.png
--- Fields ---
['image_path', 'conversation', 'mask_path', 'similar_templates', 'random_templates']
--- 1st conversation item ---
{'Question': 'Is there any defect in the object?', 'Answer': 'A', 'Options': {'A': 'Yes.', 'B': 'No.'}, 'type': 'Anomaly Detection', 'annotation': True}

✅ DataFrames created
 - img_df: (8366, 6)
 - q_df  : (39670, 7)


,key,image_path,mask_path,num_similar_templates,num_random_templates,num_questions
0,DS-MVTec/bottle/image/broken_large/000.png,DS-MVTec/bottle/image/broken_large/000.png,DS-MVTec/bottle/rbg_mask/broken_large/000_rbg_...,8,8,5
1,DS-MVTec/bottle/image/broken_large/001.png,DS-MVTec/bottle/image/broken_large/001.png,DS-MVTec/bottle/rbg_mask/broken_large/001_rbg_...,8,8,3
2,DS-MVTec/bottle/image/broken_large/002.png,DS-MVTec/bottle/image/broken_large/002.png,DS-MVTec/bottle/rbg_mask/broken_large/002_rbg_...,8,8,4


,key,question_type,question,answer,num_options,options_keys,annotation
0,DS-MVTec/bottle/image/broken_large/000.png,Anomaly Detection,Is there any defect in the object?,A,2,AB,True
1,DS-MVTec/bottle/image/broken_large/000.png,Defect Classification,There is a defect in the object. What is the t...,C,4,ABCD,True
2,DS-MVTec/bottle/image/broken_large/000.png,Defect Localization,There is a defect in the object. Where is the ...,D,4,ABCD,True
3,DS-MVTec/bottle/image/broken_large/000.png,Defect Description,There is a defect in the object. What is the a...,A,4,ABCD,True
4,DS-MVTec/bottle/image/broken_large/000.png,Defect Analysis,There is a defect in the object. What is the e...,B,4,ABCD,True



===== (A) 질문 타입 분포 =====


question_type
Anomaly Detection        8297
Defect Localization      4878
Defect Analysis          4782
Defect Description       4710
Defect Classification    4688
Object Classification    3155
Object Analysis          3084
Object Structure         3052
Object Details           3024
Name: count, dtype: int64

...

mask_path
True     5076
False    3290
Name: count, dtype: int64


===== (F) 경로 존재 여부(샘플) =====


img_exists     1.00
mask_exists    0.62
sim0_exists    1.00
rnd0_exists    1.00
Name: exist_ratio, dtype: float64


===== (G) 이미지 해상도 TOP =====


(1024, 1024)    20
(1600, 1100)     5
(800, 1600)      4
(1700, 1000)     3
(1600, 1280)     2
(840, 840)       2
(1700, 850)      1
(800, 800)       1
Name: count, dtype: int64

'GoodsAD/food_bottle/test/deformation/022_026.jpg'

## 6. 핵심 요약

- MMAD는 **객관식 멀티모달 추론 평가용 데이터셋**
- 평가 지표는 **MCQ Accuracy**
- Shot은 학습이 아니라 **추론 시 정상 reference 제공**
- mask는 보조 정보
